In [35]:
!pip install transformers --quiet
!pip install datasets --quiet
!pip install seqeval --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [69]:
import pandas as pd
# import torch 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizerFast, BertForTokenClassification, TFBertForTokenClassification
# from torch.utils.data import DataLoader
from tqdm import tqdm
# from torch.optim import SGD
from datasets import list_datasets, load_dataset, load_metric, Dataset, DatasetDict, ClassLabel, Sequence
from sklearn.metrics import classification_report

In [37]:
ds_relation = load_dataset('ade_corpus_v2', 'Ade_corpus_v2_drug_ade_relation', split='train')
ds_relation

Found cached dataset ade_corpus_v2 (/home/diana/.cache/huggingface/datasets/ade_corpus_v2/Ade_corpus_v2_drug_ade_relation/1.0.0/940d61334dbfac6b01ac5d00286a2122608b8dc79706ee7e9206a1edb172c559)


Dataset({
    features: ['text', 'drug', 'effect', 'indexes'],
    num_rows: 6821
})

In [38]:
df = ds_relation.to_pandas()
df.head()

,text,drug,effect,indexes
0,Intravenous azithromycin-induced ototoxicity.,azithromycin,ototoxicity,"{'drug': {'start_char': [12], 'end_char': [24]..."
1,"Immobilization, while Paget's bone disease was...",dihydrotachysterol,increased calcium-release,"{'drug': {'start_char': [91], 'end_char': [109..."
2,Unaccountable severe hypercalcemia in a patien...,dihydrotachysterol,hypercalcemia,"{'drug': {'start_char': [84], 'end_char': [102..."
3,METHODS: We report two cases of pseudoporphyri...,naproxen,pseudoporphyria,"{'drug': {'start_char': [58], 'end_char': [66]..."
4,METHODS: We report two cases of pseudoporphyri...,oxaprozin,pseudoporphyria,"{'drug': {'start_char': [71], 'end_char': [80]..."


In [39]:
df_relation = df['indexes'].apply(pd.Series)
df_relation[['drug_start_char', 'drug_end_char']] = df_relation.apply(lambda row: pd.Series(row['drug']), axis=1)
df_relation = df_relation.drop('drug', axis=1)
df_relation[['effect_start_char', 'effect_end_char']] = df_relation.apply(lambda row: pd.Series(row['effect']), axis=1)
df_relation = df_relation.drop('effect', axis=1)
df = pd.concat([df, df_relation], axis=1)
df = df.drop('indexes', axis=1)
df.head()

,text,drug,effect,drug_start_char,drug_end_char,effect_start_char,effect_end_char
0,Intravenous azithromycin-induced ototoxicity.,azithromycin,ototoxicity,[12],[24],[33],[44]
1,"Immobilization, while Paget's bone disease was...",dihydrotachysterol,increased calcium-release,[91],[109],[143],[168]
2,Unaccountable severe hypercalcemia in a patien...,dihydrotachysterol,hypercalcemia,[84],[102],[21],[34]
3,METHODS: We report two cases of pseudoporphyri...,naproxen,pseudoporphyria,[58],[66],[32],[47]
4,METHODS: We report two cases of pseudoporphyri...,oxaprozin,pseudoporphyria,[71],[80],[32],[47]


In [40]:
# save to JSON to then import into Dataset object
df.to_json("dataset.json", orient="records", lines=True)

In [41]:
ds_ade_relation = load_dataset("json", data_files="dataset.json", split='train')
ds_ade_relation

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/diana/.cache/huggingface/datasets/json/default-cd6a8a21d4533980/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


Dataset({
    features: ['text', 'drug', 'effect', 'drug_start_char', 'drug_end_char', 'effect_start_char', 'effect_end_char'],
    num_rows: 6821
})

In [42]:
label_list = ['O', 'B-DRUG', 'I-DRUG', 'B-EFFECT', 'I-EFFECT']
custom_seq = Sequence(feature=ClassLabel(num_classes=5, 
                                         names=label_list,
                                         names_file=None, id=None), length=-1, id=None)

ds_ade_relation.features["ner_tags"] = custom_seq
# ds_relation_tts['test'].features["ner_tags"] = custom_seq

In [43]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [44]:
t1 = tokenizer(ds_relation['text'][0], padding='max_length', max_length=25, truncation=True)
t1

{'input_ids': [101, 1130, 4487, 7912, 2285, 170, 5303, 1582, 16071, 1183, 16430, 118, 10645, 184, 2430, 2430, 8745, 9041, 119, 102, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]}

In [45]:
t1_word_id = t1.word_ids()
t1_word_id

[None,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 4,
 4,
 4,
 4,
 4,
 5,
 None,
 None,
 None,
 None,
 None,
 None]

In [46]:
MAX_SEQUENCE_LENGTH = 119

In [47]:
def generate_row_labels(row, verbose=False):
    """ Given a row from the consolidated `Ade_corpus_v2_drug_ade_relation` dataset, 
    generate BIO tags for drug and effect entities. 
    
    """

    text = row["text"]

    labels = []
    label = "O"
    prefix = ""
    bio_label = []
    
    # while iterating through tokens, increment to traverse all drug and effect spans
    drug_index = 0
    effect_index = 0
    
    tokens = tokenizer(text, return_offsets_mapping=True, padding='max_length', max_length=MAX_SEQUENCE_LENGTH)

    for n in range(len(tokens["input_ids"])):
        offset_start, offset_end = tokens["offset_mapping"][n]

        # should only happen for [CLS] and [SEP]
        if offset_end - offset_start == 0:
            labels.append(0) #-100
            continue
        
        if drug_index < len(row["drug_start_char"]) and offset_start == row["drug_start_char"][drug_index]:
            label = "DRUG"
            prefix = "B-"

        elif effect_index < len(row["effect_start_char"]) and offset_start == row["effect_start_char"][effect_index]:
            label = "EFFECT"
            prefix = "B-"
        
        labels.append(label_list.index(f"{prefix}{label}"))
        bio_label.append(f"{prefix}{label}")
            
        if drug_index < len(row["drug_end_char"]) and offset_end == row["drug_end_char"][drug_index]:
            label = "O"
            prefix = ""
            drug_index += 1
            
        elif effect_index < len(row["effect_end_char"]) and offset_end == row["effect_end_char"][effect_index]:
            label = "O"
            prefix = ""
            effect_index += 1

        # need to transition "inside" if we just entered an entity
        if prefix == "B-":
            prefix = "I-"
    
    if verbose:
        print(f"{row}\n")
        orig = tokenizer.convert_ids_to_tokens(tokens["input_ids"])
        for n in range(len(labels)):
            print(orig[n], labels[n])
    tokens["labels"] = labels
    tokens['bio_label'] = bio_label
    
    return tokens

In [48]:
labeled_dataset = ds_ade_relation.map(generate_row_labels)
labeled_dataset

Map:   0%|          | 0/6821 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'drug', 'effect', 'drug_start_char', 'drug_end_char', 'effect_start_char', 'effect_end_char', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'labels', 'bio_label'],
    num_rows: 6821
})

In [49]:
df = labeled_dataset.to_pandas()
df.head()

,text,drug,effect,drug_start_char,drug_end_char,effect_start_char,effect_end_char,input_ids,token_type_ids,attention_mask,offset_mapping,labels,bio_label
0,Intravenous azithromycin-induced ototoxicity.,azithromycin,ototoxicity,[12],[24],[33],[44],"[101, 1130, 4487, 7912, 2285, 170, 5303, 1582,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 2], [2, 5], [5, 8], [8, 11], [12,...","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 3, 4, ...","[O, O, O, O, B-DRUG, I-DRUG, I-DRUG, I-DRUG, I..."
1,"Immobilization, while Paget's bone disease was...",dihydrotachysterol,increased calcium-release,[91],[109],[143],[168],"[101, 146, 6262, 12809, 15100, 117, 1229, 8500...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 1], [1, 3], [3, 5], [5, 14], [14,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,Unaccountable severe hypercalcemia in a patien...,dihydrotachysterol,hypercalcemia,[84],[102],[21],[34],"[101, 24258, 14566, 19529, 2165, 5199, 177, 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 3], [3, 6], [6, 10], [10, 13], [1...","[0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 0, 0, 0, 0, ...","[O, O, O, O, O, B-EFFECT, I-EFFECT, I-EFFECT, ..."
3,METHODS: We report two cases of pseudoporphyri...,naproxen,pseudoporphyria,[58],[66],[32],[47],"[101, 22157, 24162, 15609, 1708, 131, 1284, 25...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 2], [2, 4], [4, 6], [6, 7], [7, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, ...","[O, O, O, O, O, O, O, O, O, O, B-EFFECT, I-EFF..."
4,METHODS: We report two cases of pseudoporphyri...,oxaprozin,pseudoporphyria,[71],[80],[32],[47],"[101, 22157, 24162, 15609, 1708, 131, 1284, 25...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 2], [2, 4], [4, 6], [6, 7], [7, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, ...","[O, O, O, O, O, O, O, O, O, O, B-EFFECT, I-EFF..."


In [50]:
len(df)

6821

In [51]:
tokenized_inputs = tokenizer('After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.', padding='max_length', max_length=50, truncation=True)
word_ids = tokenized_inputs.word_ids()
word_ids

[None,
 0,
 1,
 2,
 3,
 4,
 4,
 4,
 4,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 24,
 24,
 24,
 24,
 24,
 25,
 26,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [52]:
label = [l for i in df['bio_label'] for l in i]

unique_labels = set()
for lb in label:
    [unique_labels.add(lb)]# for i in lb if i not in unique_labels]
unique_labels

{'B-DRUG', 'B-EFFECT', 'I-DRUG', 'I-EFFECT', 'O'}

In [96]:
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}
labels_to_ids

{'I-EFFECT': 0, 'B-EFFECT': 1, 'B-DRUG': 2, 'I-DRUG': 3, 'O': 4}

In [54]:
label_all_tokens = False

def align_label(texts, labels):
  tokenized_inputs = tokenizer(texts, padding='max_length', max_length=MAX_SEQUENCE_LENGTH, truncation=True)
  word_ids = tokenized_inputs.word_ids()

  l = []
  previous = None

  for idx, i in enumerate(word_ids):
      if i is None:
          l.append(0) #-100
      elif i != previous:
          try:
              l.append(labels_to_ids[labels[idx-1]])
          except:
              l.append(0) #-100
      else:
          try:
              l.append(labels_to_ids[labels[idx-1]]) # if label_all_tokens else -100)
          except:
              l.append(0) #-100
      previous = i

  return l

In [55]:
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

In [56]:
# Train Data
train_lbl = [list(i) for i in df_train['bio_label']]
train_txt =  df_train['text'].values.tolist()
train_encodings = tokenizer(train_txt,#str(train_idx),
                                padding='max_length', 
                                max_length = MAX_SEQUENCE_LENGTH, 
                                truncation=True, 
                                return_tensors="tf") #for train_idx in train_txt]
train_labels = [align_label(i,j) for i,j in zip(train_txt, train_lbl)]

# Validation Data
val_lbl = [list(i) for i in df_val['bio_label']]
val_txt =  df_val['text'].values.tolist()
val_encodings = tokenizer(val_txt,#str(val_idx),
                                padding='max_length', 
                                max_length = MAX_SEQUENCE_LENGTH, 
                                truncation=True, 
                                return_tensors="tf") #for val_idx in val_txt]
val_labels = [align_label(i,j) for i,j in zip(val_txt, val_lbl)]

# Test Data
test_lbl = [list(i) for i in df_test['bio_label']]
test_txt =  df_test['text'].values.tolist()
test_encodings = tokenizer(test_txt,#str(test_idx),
                                padding='max_length', 
                                max_length = MAX_SEQUENCE_LENGTH, 
                                truncation=True, 
                                return_tensors="tf") #for test_idx in test_txt]
test_labels = [align_label(i,j) for i,j in zip(test_txt, test_lbl)]

In [58]:
model = TFBertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_list))

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
compute_metrics

<function __main__.compute_metrics(p)>

In [60]:
def create_bert_cls_model(max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100, 
                          num_classes = 5,
                          dropout=0.05,
                          learning_rate=0.00001):

  input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
  token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
  attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

  bert_inputs = {'input_ids': input_ids,
                 'token_type_ids': token_type_ids,
                 'attention_mask': attention_mask}

  bert_out = model(bert_inputs)

  cls_token = bert_out[0]

  last_hidden_output = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)
  last_hidden_output = tf.keras.layers.Dropout(dropout)(last_hidden_output)  

  classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(last_hidden_output)
  
  classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
  
  classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                               metrics='sparse_categorical_accuracy')

  
  return classification_model

In [61]:
bert_model = create_bert_cls_model(num_classes=len(label_list))

In [62]:
bert_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 119)]       0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 119)]        0           []                               
                                                                                                  
 token_type_ids_layer (InputLay  [(None, 119)]       0           []                               
 er)                                                                                              
                                                                                            

In [63]:
bert_model_history = bert_model.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                    np.array(train_labels),   
                                    validation_data=([val_encodings.input_ids, val_encodings.token_type_ids, val_encodings.attention_mask], 
                                    np.array(val_labels)),    
                                    batch_size=16, 
                                    epochs=5)

bert_model_history

Epoch 1/5
341/341 [==============================] - 1592s 5s/step - loss: 0.6651 - sparse_categorical_accuracy: 0.8846 - val_loss: 0.3334 - val_sparse_categorical_accuracy: 0.9479
Epoch 2/5
341/341 [==============================] - 1516s 4s/step - loss: 0.2786 - sparse_categorical_accuracy: 0.9515 - val_loss: 0.1736 - val_sparse_categorical_accuracy: 0.9580
Epoch 3/5
341/341 [==============================] - 1515s 4s/step - loss: 0.1699 - sparse_categorical_accuracy: 0.9591 - val_loss: 0.1233 - val_sparse_categorical_accuracy: 0.9629
Epoch 4/5
341/341 [==============================] - 1515s 4s/step - loss: 0.1217 - sparse_categorical_accuracy: 0.9644 - val_loss: 0.1024 - val_sparse_categorical_accuracy: 0.9634
Epoch 5/5
341/341 [==============================] - 1515s 4s/step - loss: 0.0982 - sparse_categorical_accuracy: 0.9662 - val_loss: 0.0970 - val_sparse_categorical_accuracy: 0.9621


In [66]:
bert_score = bert_model.evaluate([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask], 
                                 np.array(test_labels))

print('Test loss', bert_score[0])
print('Test accuracy', bert_score[1])

22/22 [==============================] - 54s 2s/step - loss: 0.0961 - sparse_categorical_accuracy: 0.9628
Test loss 0.09610629081726074
Test accuracy 0.9627816081047058


In [67]:
bert_predictions = bert_model.predict([test_encodings.input_ids[:], test_encodings.token_type_ids[:], test_encodings.attention_mask[:]])
bert_predictions = tf.argmax(bert_predictions, axis=-1)
bert_predictions

22/22 [==============================] - 55s 2s/step


<tf.Tensor: shape=(683, 119), dtype=int64, numpy=
array([[0, 4, 4, ..., 0, 0, 0],
       [0, 4, 4, ..., 0, 0, 0],
       [0, 4, 4, ..., 0, 0, 0],
       ...,
       [0, 4, 4, ..., 0, 0, 0],
       [0, 4, 4, ..., 0, 0, 0],
       [0, 4, 4, ..., 0, 0, 0]])>

In [57]:
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove special tokens
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [179]:
results = metric.compute(predictions=np.array(bert_predictions), references=np.array(test_labels))
results

/home/diana/anaconda3/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/diana/anaconda3/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 4 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/diana/anaconda3/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/diana/anaconda3/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/diana/anaconda3/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/diana/anaconda3/lib/python3.9/site-pack

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.9627815987302681}

In [170]:
from sklearn.metrics import f1_score

for prediction, label in zip(np.array(bert_predictions), np.array(test_labels)):
    f1 = f1_score(prediction, label, average='micro')
    print(f1)


0.970222288641859
0.9521040227930918
0.9576069744136971
0.9696038415366146
0.9832196453932397
0.9695569376526504
0.9788071326091411
0.9444899911183986
1.0
0.9475655623326442
0.9275342924054869
0.945836459878346
0.9034729681346221
1.0
1.0
1.0
0.9674136321195144
0.9657411157233977
0.9818161901632162
0.8646632846687061
0.947203525960147
0.9141321913380738
0.9001169095089016
1.0
0.9732142857142857
0.9742267051527388
0.9032108129235353
0.9516292530998414
1.0
0.9744304138767806
0.9562064825930372
0.9269689857925154
1.0
0.95180141021926
0.9357678411386853
1.0
1.0
0.9917768055089334
0.8931346123355001
1.0
0.9695691927908759
0.9660339660339661
0.9357452904826052
1.0
0.8831680040437229
0.9410561964898951
0.9263470093919921
0.9478775949364184
1.0
1.0
0.9816653934300994
0.9276532390667201
0.969930195095758
0.9004401760704281
0.9607374838480282
0.987343085382301
1.0
1.0
0.9339489237239996
0.9740247034364682
1.0
0.9572710440108247
0.9420216721924501
1.0
0.9871587786057819
0.9831932773109243
1.0
1.0


In [134]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [171]:
recall_m(np.array(bert_predictions), np.array(test_labels))

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a int64 tensor but is a float tensor [Op:AddV2]

In [152]:
tf.cast(bert_predictions, tf.int64)

<tf.Tensor: shape=(683, 119), dtype=int64, numpy=
array([[0, 4, 4, ..., 0, 0, 0],
       [0, 4, 4, ..., 0, 0, 0],
       [0, 4, 4, ..., 0, 0, 0],
       ...,
       [0, 4, 4, ..., 0, 0, 0],
       [0, 4, 4, ..., 0, 0, 0],
       [0, 4, 4, ..., 0, 0, 0]])>

In [154]:
x = K.variable([1, 2, 3, 4, 5])

# Clip the tensor values to be between 2 and 4
x_clipped = K.clip(x, 2, 4)

x_clipped

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([2., 2., 3., 4., 4.], dtype=float32)>

In [151]:
true_positives = K.sum(K.round(K.clip(tf.cast(bert_predictions, tf.int64) * test_labels, 0, 1)))
possible_positives = K.sum(K.round(K.clip(tf.cast(bert_predictions, tf.int64), 0, 1)))
true_positives / (possible_positives + K.epsilon())

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a int64 tensor but is a float tensor [Op:AddV2]

In [175]:
def create_bert_cls_model2(max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100, 
                          num_classes = 5,
                          dropout=0.05,
                          learning_rate=0.00001):

    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                 'token_type_ids': token_type_ids,
                 'attention_mask': attention_mask}

    bert_out = model(bert_inputs)

    cls_token = bert_out[0]

    last_hidden_output = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)
    last_hidden_output = tf.keras.layers.Dropout(dropout)(last_hidden_output)  

    classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(last_hidden_output)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                               metrics=['sparse_categorical_accuracy', tf.keras.metrics.Recall()])


    return classification_model

In [176]:
bert_model2 = create_bert_cls_model2(num_classes=len(label_list))

In [177]:
bert_model2.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 119)]       0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 119)]        0           []                               
                                                                                                  
 token_type_ids_layer (InputLay  [(None, 119)]       0           []                               
 er)                                                                                              
                                                                                            

In [178]:
bert_model_history2 = bert_model2.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                    np.array(train_labels),   
                                    validation_data=([val_encodings.input_ids, val_encodings.token_type_ids, val_encodings.attention_mask], 
                                    np.array(val_labels)),    
                                    batch_size=16, 
                                    epochs=5)

bert_model_history2

Epoch 1/5


ValueError: in user code:

    File "/home/diana/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/diana/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/diana/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/diana/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1028, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/diana/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1122, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/diana/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 605